In [ ]:
import pandas as pd
from safetensors import torch
import torch

In [ ]:
train_set = pd.read_csv('train.csv')
test_set = pd.read_csv('test.csv')

In [ ]:
test_set

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.
...,...,...
153159,fffcd0960ee309b5,". \n i totally agree, this stuff is nothing bu..."
153160,fffd7a9a6eb32c16,== Throw from out field to home plate. == \n\n...
153161,fffda9e8d6fafa9e,""" \n\n == Okinotorishima categories == \n\n I ..."
153162,fffe8f1340a79fc2,""" \n\n == """"One of the founding nations of the..."


In [ ]:
train_set.iloc[:, 2:].sum().sum() / len(train_set) * 100

21.99522469621673

In [ ]:
from transformers import BertTokenizer, Trainer, BertForSequenceClassification, TrainingArguments

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
X_train = train_set['comment_text'].tolist()
train_encoding = tokenizer(X_train, padding=True, truncation=True, max_length=64)

In [ ]:
X_test = test_set['comment_text'].tolist()
test_encoding = tokenizer(X_test, padding=True, truncation=True, max_length=64)

In [ ]:
y_train = train_set.iloc[:, 2:].max(axis=1).tolist()

In [ ]:
print(train_encoding.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


In [ ]:
print(train_encoding['attention_mask'][0])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
print(len(train_encoding['input_ids']), len(y_train))
print(y_train[:5])

159571 159571
[0, 0, 0, 0, 0]


In [ ]:
class ToxicDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

In [ ]:
class ToxicTestDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __len__(self):
        return len(self.encodings)
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

In [ ]:
train_dataset = ToxicDataset(train_encoding, y_train)
test_dataset = ToxicTestDataset(test_encoding)

In [ ]:
test_dataset[0]

{'input_ids': tensor([  101, 10930,  7743, 14855,  3627,  2003,  2062, 10514,  9468,  2229,
          3993,  2059,  2017,  1005,  2222,  2412,  2022,  2054,  2015,  2039,
          2007,  2017,  1998, 22650,  2017,  6517,  9587, 11263,  3600,  3022,
          1012,  1012,  1012,  1045,  2323,  7743, 14308, 24471,  9004,  9072,
          2594,  2317,  5344,  1998,  2131,  2017,  2000,  3610,  2026,  4632,
          2017,  4364,  5305,  2368,  2033,  1012, 14855,  3627,  2003,  2055,
          6620,  1999,  4830,   102]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir='./output',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    logging_dir='./logs',
    logging_steps=10,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [ ]:
import os
os.environ["WANDB_MODE"] = "dryrun"

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model.to(device)

Using device: cuda


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.112000,No log
2,0.109400,No log
3,0.081000,No log


Epoch,Training Loss,Validation Loss
1,0.112000,No log
2,0.109400,No log
3,0.081000,No log
4,0.065200,No log
5,0.047700,No log


TrainOutput(global_step=24935, training_loss=0.06541186308595444, metrics={'train_runtime': 8223.4943, 'train_samples_per_second': 97.021, 'train_steps_per_second': 3.032, 'total_flos': 2.62405588842816e+16, 'train_loss': 0.06541186308595444, 'epoch': 5.0})

In [ ]:
model.save_pretrained("bert_toxic_model")
tokenizer.save_pretrained("bert_toxic_model")

('bert_toxic_model/tokenizer_config.json',
 'bert_toxic_model/special_tokens_map.json',
 'bert_toxic_model/vocab.txt',
 'bert_toxic_model/added_tokens.json')

In [ ]:
from google.colab import files
import shutil

shutil.make_archive("bert_toxic_model", 'zip', "bert_toxic_model")
files.download("bert_toxic_model.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>